In [5]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super(LSTM, self).__init__()
    self.M = hidden_dim
    self.L = layer_dim

    self.rnn = nn.LSTM(
        input_size=input_dim,
        hidden_size=hidden_dim,
        num_layers=layer_dim,
        batch_first=True)
    #batch_first to have (batch_dim, seq_dim, feature_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, X):
    # initial hidden state and cell state
    h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
    c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

    out, (hn, cn) = self.rnn(X, (h0.detach(), c0.detach()))

    # h(T) at the final time step
    out = self.fc(out[:, -1, :])
    return out

model = LSTM(12, 512, 2, 1)
model.load_state_dict(torch.load('LSTM_model.pth', map_location=torch.device('cpu')))
model.eval()

LSTM(
  (rnn): LSTM(12, 512, num_layers=2, batch_first=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [6]:

#Reading data 
weather = pd.read_csv("Input.csv", on_bad_lines='skip')
weather = weather[['Aasta','Kuu','Päev','Kell (UTC)','Tunni keskmine summaarne kiirgus W/m²','Õhurõhk merepinna kõrgusel hPa','Õhurõhk jaama kõrgusel hPa','Tunni sademete summa mm','Suhteline õhuniiskus %','Õhutemperatuur °C','Tunni miinimum õhutemperatuur °C','Tunni maksimum õhutemperatuur °C','10 minuti keskmine tuule suund °', '10 minuti keskmine tuule kiirus m/s', 'Tunni maksimum tuule kiirus m/s']]

# Combine 'Aasta', 'Kuu', 'Päev' and 'Kell (UTC)' into a datetime
weather['Date Time'] = pd.to_datetime(weather['Aasta'].astype(str) + weather['Kuu'].astype(str).str.zfill(2) + weather['Päev'].astype(str).str.zfill(2) + weather['Kell (UTC)'], format='%Y%m%d%H:%M')
date_time = weather['Date Time']
# Remove the 'Date Time' column
weather = weather.drop(['Aasta', 'Kuu', 'Päev', 'Kell (UTC)'], axis=1)

weather = weather.rename(columns={'Õhutemperatuur °C': 'temperature', 'Date Time': 'date', 'Tunni keskmine summaarne kiirgus W/m²' : 'radiation','Õhurõhk merepinna kõrgusel hPa' : 'pressure','Õhurõhk jaama kõrgusel hPa': 'pressure2','Tunni sademete summa mm' : 'precip','Suhteline õhuniiskus %' : 'humidity','Tunni miinimum õhutemperatuur °C' : 'tempmin','Tunni maksimum õhutemperatuur °C': 'tempmax','10 minuti keskmine tuule suund °' : 'winddir', '10 minuti keskmine tuule kiirus m/s' : 'windspeed', 'Tunni maksimum tuule kiirus m/s': 'windmax'})
weather['date'] = pd.to_datetime(weather['date'])
weather.set_index('date')

weather.dropna(inplace=True)


In [7]:
# initial hidden state and cell state
#h0 = torch.zeros((model.L, new_data.size(0), model.M)).to(device)
#c0 = torch.zeros((model.L, new_data.size(0), model.M)).to(device)


In [24]:
# Select the features for prediction
# Convert the data to a numpy array with float type
data = data.astype(float)

# Create a tensor from the data
data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(0)

# Move the tensor to the device
data_tensor = data_tensor.to(device)

# Make predictions
predictions = model(data_tensor)

# Convert the predictions tensor to a numpy array
predictions = predictions.detach().cpu().numpy()

# Denormalize the predictions
predictions = (predictions * data.std()) + data.mean()

# Check if predictions tensor has enough dimensions
if predictions.ndim > 1:
    # Get the predicted temperature, tempmin, and tempmax
    predicted_temperature = predictions[0, 0]
    predicted_tempmin = predictions[0, 1]
    predicted_tempmax = predictions[0, 2]
else:
    # Handle the case when predictions tensor has only one dimension
    predicted_temperature = predictions[0]
    predicted_tempmin = predictions[0]
    predicted_tempmax = predictions[0]


IndexError: index 1 is out of bounds for axis 1 with size 1